In [1]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

%load_ext autoreload
%autoreload 2

# Dataset creation

In [2]:
from src.dataset import TDMSDataset, PATH

dataset = TDMSDataset(PATH.TRAIN)

In [3]:
from src.content_extraction import naive_doctaet

doctaets = []
for i in range(len(dataset)):
    f, tex, jsn = dataset[i]
    doctaet = naive_doctaet(tex)
    doctaets.append({"f": f, "doctaet": doctaet, "annotation": jsn})


In [4]:
import pandas as pd

df = pd.DataFrame(doctaets)


df

,f,doctaet,annotation
0,0704.1707,title\nA Cut-free Sequent Calculus for\\ Bi-I...,unanswerable\n
1,0704.2295,title\nUsing Image Attributes for Human Identi...,unanswerable\n
2,0704.3238,title\nAlternative axiomatics and complexity o...,unanswerable\n
3,0705.0599,title\nNodeTrix: Hybrid Representation for Ana...,unanswerable\n
4,0705.0742,title\nMIMO detection employing Markov Chain M...,unanswerable\n
...,...,...,...
12283,2312.02139v1,table\n[t]\n\caption{FID performance compariso...,"[{'LEADERBOARD': {'Task': 'Image Generation', ..."
12284,2312.02185v1,title\nVirtual Fusion with Contrastive Learnin...,[{'LEADERBOARD': {'Task': 'Human Activity Reco...
12285,2312.03288v1,title\nSTEP CATFormer: Spatial-Temporal Effect...,[{'LEADERBOARD': {'Task': 'Skeleton Based Acti...
12286,2312.03430v1,title\nShareCMP: Polarization-Aware RGB-P Sema...,[{'LEADERBOARD': {'Task': 'Semantic Segmentati...


In [5]:
# from huggingface_hub import login

# login()

In [6]:
# cut doctaets

from transformers import AutoTokenizer
from src.prompt_templates import few_shot_template_initial

model = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model)

context_len = 8192
prompt_len = len(tokenizer(few_shot_template_initial(""))["input_ids"])


prompt_len

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


290

In [7]:
allowed_len = context_len - prompt_len - 150 # Buffer
allowed_len

# assumption: 3 chars per token average
num_chars_allowed = allowed_len * 3
num_chars_allowed

23256

In [8]:
from src.content_extraction import naive_doctaet

prompt_template = few_shot_template_initial

data = []
for i in range(len(dataset)):

    f, tex, jsn = dataset[i]
    doctaet = naive_doctaet(tex)
    doctaet_cut = doctaet[:min(len(doctaet), num_chars_allowed)]
    result = {
        "prompt": prompt_template(doctaet_cut),
        "annotation": str(jsn)
    }
    data.append(result)


In [9]:
import pandas as pd

df = pd.DataFrame(data)

In [10]:
df.to_feather("finetune_dataset_few_shot_initial.feather")